# 1. Data

## 1.1 Loading and combining the data together

In [ ]:

from Helper_functions import load_and_combine_parquet_files, average_the_values, apply_kalman_filter, parallel_kalman_filter

In [ ]:

import os
directory_path = os.getcwd()
combined_results = load_and_combine_parquet_files(directory_path)

napetost_all_df = combined_results["napetost_all"]
napetost_wavelet_df = combined_results["napetost_wavelet"]
tok_all_df = combined_results["tok_all"]
tok_wavelet_df = combined_results["tok_wavelet"]

## 1.2 Checking the sampling frequencies

In [ ]:
# Calculate the total number of data points for each file
data_points_per_file = napetost_wavelet_df.groupby(level=0).size()

# Calculate the sampling frequency for each file
sampling_frequency_per_file = data_points_per_file / 900

# Assign the sampling frequency to a new column in the DataFrame
napetost_wavelet_df['sampling frequency'] = sampling_frequency_per_file.reindex(napetost_wavelet_df.index, level=0)
# Find unique sampling frequencies and filter the DataFrame to keep only unique ones
unique_sampling_frequencies = napetost_wavelet_df['sampling frequency'].drop_duplicates()
# Filter the original DataFrame to show rows with unique sampling frequencies
unique_frequency_df = napetost_wavelet_df[napetost_wavelet_df['sampling frequency'].isin(unique_sampling_frequencies)]

# Display the full rows for unique sampling frequencies
print(unique_sampling_frequencies.to_markdown())

# 2. Extracting useful information 

## 2.1 Avreaging values 

In [ ]:
average_napetost_all_df = average_the_values(napetost_all_df, x_col="napetost_x", y_col="napetost_y", interval_duration_seconds = 5)
average_napetost_all_df
apply_kalman_filter(average_napetost_all_df, x_col="x_values", y_col="averaged")

In [ ]:
import plotly.graph_objects as go

# Select the first 10 unique files
first_10_files = average_napetost_all_df.index.unique()
filtered_df = average_napetost_all_df[average_napetost_all_df.index.isin(first_10_files)]

# Create a Plotly figure
fig = go.Figure()

# Add a trace for each measurement file
for file_name in filtered_df.index.unique():
    file_data = filtered_df[filtered_df.index == file_name]
    fig.add_trace(go.Scatter(
        x=file_data['interval_bin'],
        y=file_data['averaged'],
        mode='lines+markers',
        name=file_name
    ))

# Update layout for readability and height
fig.update_layout(
    title="30-Second Average Y-Values for First 10 Measurements",
    xaxis_title="Interval Bin (ticks)",
    yaxis_title="30-Second Average Y-Value",
    template="plotly_white",
    legend_title="Measurement Files",
    height=1500  # Increase height for a tall plot
)

# Show the plot
fig.show()


In [ ]:
import plotly.graph_objects as go

# Select the first 10 unique files
first_10_files = average_napetost_all_df.index.unique()
filtered_df = average_napetost_all_df[average_napetost_all_df.index.isin(first_10_files)]

# Create a Plotly figure
fig = go.Figure()

# Add a trace for each measurement file
for file_name in filtered_df.index.unique():
    file_data = filtered_df[filtered_df.index == file_name]
    fig.add_trace(go.Scatter(
        x=file_data['interval_bin'],
        y=file_data['kalman_filtered'],
        mode='lines+markers',
        name=file_name
    ))

# Update layout for readability and height
fig.update_layout(
    title="30-Second Average Y-Values for First 10 Measurements",
    xaxis_title="Interval Bin (ticks)",
    yaxis_title="30-Second Average Y-Value",
    template="plotly_white",
    legend_title="Measurement Files",
    height=1500  # Increase height for a tall plot
)

# Show the plot
fig.show()
